# Census release 2023-12-15 (LTS)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import lamindb as ln
import lnschema_bionty as lb
import pandas as pd
from cellxgene_lamin import get_datasets_from_cxg, get_collections_from_cxg

ln.settings.verbosity = "hint"

2024-01-15 07:01:24,977:INFO - NumExpr defaulting to 2 threads.


💡 lamindb instance: laminlabs/cellxgene


In [3]:
census_version = "2023-12-15"

In [4]:
s3path = f"s3://cellxgene-data-public/cell-census/{census_version}/h5ads"
ln.UPath(s3path)

S3Path('s3://cellxgene-data-public/cell-census/2023-12-15/h5ads')

In [5]:
ln.UPath(s3path).view_tree()

h5ads (0 sub-directories & 1113 files with suffixes '.h5ad'): 
├── 00099d5e-154f-4a7a-aa8d-fa30c8c0c43c.h5ad
├── 0041b9c3-6a49-4bf7-8514-9bc7190067a7.h5ad
├── 00476f9f-ebc1-4b72-b541-32f912ce36ea.h5ad
├── 00e5dedd-b9b7-43be-8c28-b0e5c6414a62.h5ad
├── 00ff600e-6e2e-4d76-846f-0eec4f0ae417.h5ad
├── 01209dce-3575-4bed-b1df-129f57fbc031.h5ad
...


In [6]:
ln.track()

💡 notebook imports: cellxgene_lamin==0.0.1 lamindb==0.67.1 lnschema_bionty==0.39.0 pandas==2.1.4
💡 loaded: Transform(uid='G69jtgzKO0eJ6K79', name='Census release 2023-12-15 (LTS)', short_name='cencus-release-2023-12-15-LTS', version='0', type='notebook', updated_at=2024-01-11 09:33:35 UTC, created_by_id=1)
💡 loaded: Run(uid='UAAiLAi0BrLvlKnsuvP3', run_at=2024-01-15 07:01:32 UTC, transform_id=17, created_by_id=1)


💡 tracked pip freeze > /home/sagemaker-user/.cache/lamindb/run_env_pip_UAAiLAi0BrLvlKnsuvP3.txt
💡   parent transform: Transform(uid='V4AGIdOJcOgj6K79', name='Census release 2023-12-15 (LTS)', short_name='cencus-release-2023-12-15-LTS', version='0', type='notebook', updated_at=2024-01-11 09:06:31 UTC, created_by_id=1)


Get all datasets and associated metadata using cellxgene REST API:

In [7]:
cxg_datasets = get_datasets_from_cxg()
len(cxg_datasets)

1163

In [8]:
cxg_datasets[0].keys()

dict_keys(['assay', 'assets', 'cell_count', 'cell_type', 'citation', 'collection_doi', 'collection_id', 'collection_name', 'collection_version_id', 'dataset_id', 'dataset_version_id', 'development_stage', 'disease', 'donor_id', 'explorer_url', 'is_primary_data', 'mean_genes_per_cell', 'organism', 'primary_cell_count', 'processing_status', 'published_at', 'revised_at', 'schema_version', 'self_reported_ethnicity', 'sex', 'suspension_type', 'tissue', 'title', 'tombstone', 'x_approximate_distribution'])

## Register artifacts

In [18]:
artifacts = ln.Artifact.from_dir(s3path)
ln.save(artifacts)

❗ this creates one artifact per file in the directory - you might simply call ln.Artifact(dir) to get one artifact for the entire directory


In [9]:
artifacts = ln.Artifact.filter(key__contains = census_version).all()
len(artifacts)

1113

In [15]:
for cxg_dataset in cxg_datasets:
    artifact = artifacts.filter(key__contains=cxg_dataset["dataset_id"]).one_or_none()
    if artifact is not None:
        artifact.n_observations = cxg_dataset["cell_count"]
        artifact.description = cxg_dataset["title"]
        artifact.save()

In [196]:
collection = ln.Collection(artifacts, name="cellxgene-census", version=census_version)
collection.save()

## Register collections

In [32]:
cxg_collections = get_collections_from_cxg()

In [195]:
for collection_meta in cxg_collections:
    keys = [f'cell-census/{census_version}/h5ads/{dataset["dataset_id"]}.h5ad' for dataset in collection_meta["datasets"]]
    collection_artifacts = artifacts.filter(key__in = keys).all()
    if collection_artifacts.count() > 0:
        kwargs = dict(
            name=collection_meta["name"], 
            description=collection_meta["doi"],
            reference=collection_meta["collection_id"],
            reference_type="CELLxGENE Collection ID",
            version=census_version,
        )
        collection_record = ln.Collection(
            collection_artifacts,
            **kwargs,
        )
        # if is needed here as .save() errors if collection is already saved
        if collection_record._state.adding:
            collection_record.save()

## Register metadata

### Register new features and parent labels

In [9]:
from cellxgene_lamin._features import OBS_FEATURES, EXT_FEATURES, register_feature_set

In [15]:
obs_feature_set = ln.FeatureSet.filter(name="obs features").one_or_none()
if obs_feature_set is None:
    obs_feature_set = register_feature_set(artifacts, "obs")

ext_feature_set = ln.FeatureSet.filter(name="external metadata").one_or_none()
if ext_feature_set is None:
    ext_feature_set = register_feature_set(artifacts, "ext")

In [16]:
features = ln.Feature.lookup()

## organisms

In [18]:
from cellxgene_lamin._organism import register_organisms, annotate_organisms

In [ ]:
register_organisms(cxg_datasets)

Link collections and organisms to artifacts:

In [26]:
annotate_organisms(artifacts, cxg_datasets)

## ontologies

Register all ontology ids:

In [21]:
from cellxgene_lamin._ontology import create_ontology_record_from_source, register_ontology_ids

In [22]:
register_ontology_ids(cxg_datasets)

registering assay
registering cell_type
✅ loaded 752 CellType records matching ontology_id: 'CL:0019003', 'CL:1000494', 'CL:0000708', 'CL:0009089', 'CL:1000850', 'CL:0009111', 'CL:0000055', 'CL:1000309', 'CL:0000890', 'CL:1001045', 'CL:0000394', 'CL:0000553', 'CL:0000123', 'CL:0002145', 'CL:0002085', 'CL:1000223', 'CL:1000692', 'CL:0000084', 'CL:0000485', 'CL:0000066', ...
✅ created 20 CellType records from Bionty matching ontology_id: 'CL:0000459', 'CL:0000753', 'CL:0000754', 'CL:0000756', 'CL:0000757', 'CL:0000758', 'CL:0000759', 'CL:0000760', 'CL:0000761', 'CL:0000917', 'CL:0002102', 'CL:0002626', 'CL:0002672', 'CL:0004213', 'CL:0004214', 'CL:0004215', 'CL:0004216', 'CL:0011031', 'CL:1000042', 'CL:4023161'
❗ now recursing through parents: this only happens once, but is much slower than bulk saving
💡 you can switch this off via: lb.settings.auto_save_parents = False
💡 also saving parents of CellType(uid='15Fmigot', name='noradrenergic cell', ontology_id='CL:0000459', synonyms='noradr

## donors and suspension_types

In [39]:
from cellxgene_lamin._labels import register_ulabels

In [40]:
register_ulabels(cxg_datasets, "donor_id")

registered 205 donor_ids


In [41]:
register_ulabels(cxg_datasets, "suspension_type")

registered 0 suspension_types


## Annotate artifacts with obs metadata

In [45]:
from cellxgene_lamin._features import FEATURE_TO_ACCESSOR

In [47]:
features = ln.Feature.lookup()

for idx, cxg_dataset in enumerate(cxg_datasets):
    if idx % 100 == 0:
        print(f"annotating dataset {idx} of {len(cxg_datasets)}")
    artifact = artifacts.filter(key__contains=cxg_dataset["dataset_id"]).one_or_none()
    if artifact is None:
        continue
    for field, terms in cxg_dataset.items():
        if field not in FEATURE_TO_ACCESSOR:
            continue
        accessor, orm = FEATURE_TO_ACCESSOR.get(field)
        if field in ["donor_id", "suspension_type", "tissue_type"]:
            records = orm.from_values(terms, field="name")
            if len(records) > 0:
                # stratify by feature so that link tables records are written
                artifact.labels.add(records, feature=getattr(features, field))
        else:
            records = orm.from_values(
                [i["ontology_term_id"] for i in terms], field="ontology_id"
            )
            if len(records) > 0:
                getattr(artifact, accessor).add(*records)

# clean up the 2 "unknowns" in DevelopmentalStage
lb.DevelopmentalStage.filter(name="unknown").exclude(ontology_id="unknown").delete()

annotating dataset 0 of 1163
annotating dataset 100 of 1163
annotating dataset 200 of 1163
annotating dataset 300 of 1163
annotating dataset 400 of 1163
annotating dataset 500 of 1163
annotating dataset 600 of 1163
annotating dataset 700 of 1163
annotating dataset 800 of 1163
annotating dataset 900 of 1163
annotating dataset 1000 of 1163
annotating dataset 1100 of 1163


## Validate and register genes

In [52]:
from cellxgene_lamin._gene import register_genes

Register all genes for each organism:

In [34]:
register_genes()

registering homo_sapiens genes
❗ did not create Gene records for 147 non-validated ensembl_gene_ids: 'ENSG00000112096', 'ENSG00000137808', 'ENSG00000161149', 'ENSG00000182230', 'ENSG00000203812', 'ENSG00000204092', 'ENSG00000205485', 'ENSG00000212951', 'ENSG00000215271', 'ENSG00000221995', 'ENSG00000224739', 'ENSG00000224745', 'ENSG00000225178', 'ENSG00000225932', 'ENSG00000226377', 'ENSG00000226380', 'ENSG00000226403', 'ENSG00000227021', 'ENSG00000227220', 'ENSG00000227902', ...
❗ 147 terms (0.20%) are not validated for ensembl_gene_id: ENSG00000269933, ENSG00000261737, ENSG00000259834, ENSG00000256374, ENSG00000263464, ENSG00000203812, ENSG00000272196, ENSG00000272880, ENSG00000284299, ENSG00000270188, ENSG00000287116, ENSG00000237133, ENSG00000224739, ENSG00000227902, ENSG00000239467, ENSG00000272551, ENSG00000280374, ENSG00000284741, ENSG00000236886, ENSG00000229352, ...
registering mus_musculus genes
❗ did not create Gene records for 135 non-validated ensembl_gene_ids: 'ENSMUSG000

## Link metadata to individual artifacts

annotate with genes measured in each artifact:

In [53]:
organisms = lb.Organism.lookup(field=lb.Organism.scientific_name)

In [ ]:
for idx, artifact in enumerate(artifacts):
    if idx % 100 == 0:
        print(f"annotating dataset {idx} of {len(artifacts)}")

    adata_backed = artifact.backed()
    var_names = adata_backed.var_names
    organism_record = artifact.organism.first()
    if organism_record is None:
        print(f"No organism found for artifact: {artifact}")
        continue
    genes = lb.Gene.from_values(
        var_names, field=lb.Gene.ensembl_gene_id, organism=organism_record
    )

    if len(genes) == 0 and var_names[0].startswith("ENSG"):
        genes += lb.Gene.from_values(
            var_names, field=lb.Gene.ensembl_gene_id, organism="human"
        )

    if len(var_names[var_names.str.startswith("ERCC")]) > 0:
        genes += lb.Gene.from_values(
            var_names,
            field=lb.Gene.ensembl_gene_id,
            organism=organisms.synthetic_construct,
        )
    if len(var_names[var_names.str.startswith("ENSSASG")]) > 0:
        genes += lb.Gene.from_values(
            var_names,
            field=lb.Gene.ensembl_gene_id,
            organism=organisms.severe_acute_respiratory_syndrome_coronavirus_2,
        )

    var_feature_set_artifact = ln.FeatureSet(genes, type="number")
    var_feature_set_artifact.save()
    artifact.feature_sets.add(var_feature_set_artifact, through_defaults={"slot": "var"})

annotating dataset 0 of 1113
✅ loaded: FeatureSet(uid='qztydaKUhnRp4zHzXUXd', n=30933, type='number', registry='bionty.Gene', hash='bMTbQNmrQxHwjX4k3XGh', updated_at=2024-01-15 08:00:19 UTC, created_by_id=1)
✅ loaded: FeatureSet(uid='4a3TNgtW7Gqo8rjmF6To', n=33234, type='number', registry='bionty.Gene', hash='QxtBB8YlF4P9hZsWN1iG', updated_at=2024-01-15 08:02:07 UTC, created_by_id=1)
✅ loaded: FeatureSet(uid='yJkW1pbCGyfFdJ32yvbf', n=27651, type='number', registry='bionty.Gene', hash='aALn7EGvPqdoBJdhQ9QZ', updated_at=2024-01-15 08:01:41 UTC, created_by_id=1)
✅ loaded: FeatureSet(uid='yJkW1pbCGyfFdJ32yvbf', n=27651, type='number', registry='bionty.Gene', hash='aALn7EGvPqdoBJdhQ9QZ', updated_at=2024-01-15 08:04:03 UTC, created_by_id=1)
❗ loading non-default source inside a LaminDB instance
💡 please consider:
    close your instance via `lamin close` and use Bionty stand alone
    OR
    modify currently_used Gene source in `lnschema_bionty.PublicSource`
❗ no Bionty source found, skippin

In [55]:
artifact.describe()

Artifact(uid='4UdE334dCgOh0n5xk4ix', key='cell-census/2023-12-15/h5ads/108c2b65-4ea9-4792-8b5b-451842faf35b.h5ad', suffix='.h5ad', accessor='AnnData', description='Massively multiplex chemical transcriptomics at single-cell resolution - A549', size=342294033, hash='XHxCWnmWNjKufUTzMOeWCw-41', hash_type='md5-n', n_observations=143015, visibility=1, key_is_virtual=False, updated_at=2024-01-11 09:43:51 UTC)

Provenance:
  🗃️ storage: Storage(uid='oIYGbD74', root='s3://cellxgene-data-public', type='s3', region='us-west-2', updated_at=2023-10-16 15:04:08 UTC, created_by_id=1)
  📔 transform: Transform(uid='V4AGIdOJcOgj6K79', name='Census release 2023-12-15 (LTS)', short_name='cencus-release-2023-12-15-LTS', version='0', type='notebook', updated_at=2024-01-11 09:06:31 UTC, created_by_id=1)
  👣 run: Run(uid='UAAiLAi0BrLvlKnsuvP3', run_at=2024-01-15 07:01:32 UTC, transform_id=17, created_by_id=1)
  👤 created_by: User(uid='kmvZDIX9', handle='sunnyosun', name='Sunny Sun', updated_at=2023-12-13 16

## Annotate tissue_type

Before CxG schema 4.0, tissue_type column was not annotated, instead "cell culture" or "organoid" was added to the record ontology_id.

In [57]:
register_ulabels(cxg_datasets, "tissue_type")

In [58]:
is_tissue_type = ln.ULabel.filter(name="is_tissue_type").one()
tissue_types = is_tissue_type.children.lookup()
features = ln.Feature.lookup()

In [59]:
organoids = lb.Tissue.filter(ontology_id__contains="organoid").all()
organoids.df()

,uid,name,ontology_id,abbr,synonyms,description,created_at,updated_at,public_source_id,created_by_id
id,,,,,,,,,,


In [61]:
for record in organoids:
    print(record.name)
    ontology_id = record.ontology_id.split(" ")[0]
    tissue_record = lb.Tissue.from_bionty(ontology_id=ontology_id)
    if tissue_record._state.adding:
        tissue_record.save()
    for f in tissue_record.artifacts.all():
        f.labels.add(tissue_types.organoid, features.tissue_type)

In [62]:
organoids.delete()

In [63]:
cell_cultures = lb.Tissue.filter(ontology_id__contains="cell culture").all()
cell_cultures.df()

,uid,name,ontology_id,abbr,synonyms,description,created_at,updated_at,public_source_id,created_by_id
id,,,,,,,,,,


In [64]:
for record in cell_cultures:
    print(record.name)
    ontology_id = record.ontology_id.split(" ")[0]
    tissue_record = lb.CellType.from_bionty(ontology_id=ontology_id)
    if tissue_record._state.adding:
        tissue_record.save()
    for f in tissue_record.artifacts.all():
        f.labels.add(tissue_types.cell_culture, features.tissue_type)

In [65]:
cell_cultures.delete()

## Link metadata to collection

In [74]:
collection = ln.Collection.filter(name="cellxgene-census", version=census_version).one()

feature sets:

In [75]:
collection.feature_sets.add(
    ln.FeatureSet.filter(name__contains="obs").one(), through_defaults={"slot": "obs"}
)
collection.feature_sets.add(
    ln.FeatureSet.filter(name__contains="ext").one(),
    through_defaults={"slot": "external"},
)

In [76]:
collection.describe()

Collection(uid='T9xK6Iu5mJCY0lmd832u', name='cellxgene-census', version='2023-12-15', hash='0NB32iVKG5ttaW5XILvG', visibility=1, updated_at=2024-01-11 13:47:39 UTC)

Provenance:
  📔 transform: Transform(uid='G69jtgzKO0eJ6K79', name='Census release 2023-12-15 (LTS)', short_name='cencus-release-2023-12-15-LTS', version='0', type='notebook', updated_at=2024-01-11 09:33:35 UTC, created_by_id=1)
  👣 run: Run(uid='UAAiLAi0BrLvlKnsuvP3', run_at=2024-01-15 07:01:32 UTC, transform_id=17, created_by_id=1)
  👤 created_by: User(uid='kmvZDIX9', handle='sunnyosun', name='Sunny Sun', updated_at=2023-12-13 16:23:44 UTC)
Features:
  obs: FeatureSet(uid='zAQ6WnmIMDLslhfgdIOt', name='obs metadata', n=10, type='category', registry='core.Feature', hash='CFxuf-VqTFbrkbqPHiY-', updated_at=2023-12-09 16:16:49 UTC, created_by_id=1)
    🔗 tissue (0, bionty.Tissue): 
    🔗 tissue_type (0, core.ULabel): 
    🔗 assay (0, bionty.ExperimentalFactor): 
    🔗 cell_type (0, bionty.CellType): 
    🔗 development_stage (0